In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya

/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [26]:
files = ['test-translated-cnn-daily.jsonl', 
         'train-translated-cnn-daily.jsonl'
        ]

In [4]:
import json

with open(files[0]) as fopen:
    for l in fopen:
        data = json.loads(l)

In [6]:
maxlen = 900

In [14]:
data['translated_highlight']

'Peter Morris hancur ketika adiknya, Claire, meninggal dalam kemalangan kereta. Hanya setahun sebelumnya, dia telah berkahwin dengan Malcolm Webster. Dia percaya adiknya telah meninggal dalam kemalangan tragis. Perlu dua dekad kebenaran untuk dinyatakan. Claire dibunuh oleh Webster sehingga dia dapat memperoleh insurans hayatnya. Berusaha melakukan perkara yang sama kepada isteri keduanya pada tahun 1999 - dan dipenjara pada tahun 2011.'

In [18]:
def cut_article(article, maxlen = 900):
    splitted = malaya.text.function.split_into_sentences(article, minimum_length = 2)
    temp = []
    for s in splitted:
        temp.append(s)
        if len(' '.join(temp).split()) > maxlen:
            break
    
    return ' '.join(temp)

In [19]:
cut_article(data['article'])

"Peter Morris was devastated when his sister, Claire, died in a car crash when she was just 32. Only a year before, he had walked her down the aisle to give her away to her husband, Malcolm Webster, from Surrey, then aged 33, who seemed equally bereft at the loss of his wife. Peter, from Gillingham, Kent, believed Claire had died in a tragic accident that happened in Aberdeen, Scotland, in 1994. Scroll down for video . Peter Morris, left, was devastated when his sister, Claire, pictured right on her hen do, died in a car crash . It took two decades for the truth to be revealed - Claire had in fact been murdered by Webster so he could cash in on her life insurance. Webster had drugged his wife at their home in the Scottish village of Tarves, where they had settled after their wedding, before taking her for a drive. He then deliberately crashed their car and, after making his own escape, set it on fire so Claire had no hope of survival. Last year Claire's story was turned into an ITV dra

In [20]:
cut_article(data['translated_article'])

'Peter Morris hancur ketika adiknya, Claire, meninggal dalam kemalangan kereta ketika dia baru berusia 32 tahun. Hanya setahun sebelumnya, dia telah berjalan menyusuri lorong untuk memberikannya kepada suaminya, Malcolm Webster, dari Surrey, yang ketika itu berusia 33 tahun, yang sepertinya sama-sama bingung dengan kehilangan isterinya. Peter, dari Gillingham, Kent, percaya Claire telah meninggal dalam kemalangan tragis yang berlaku di Aberdeen, Scotland, pada tahun 1994. Tatal ke bawah untuk video. Peter Morris, kiri, hancur ketika adiknya, Claire, yang digambarkan tepat di ayamnya, meninggal dalam kemalangan kereta. Perlu dua dekad kebenaran untuk dinyatakan - Claire sebenarnya telah dibunuh oleh Webster sehingga dia dapat memperoleh insurans hayatnya. Webster telah meminum isterinya di rumah mereka di kampung Tarves di Scotland, di mana mereka telah menetap setelah perkahwinan mereka, sebelum membawanya untuk memandu. Dia kemudian sengaja merempuh kereta mereka dan, setelah melarika

In [27]:
from tqdm import tqdm

with open('train.json', 'w') as fopen_jsonl:
    for f in files:
        print(f)
        with open(f) as fopen:
            for l in tqdm(fopen):
                data = json.loads(l)
                en = cut_article(data['article'])
                ms = cut_article(data['translated_article'])
                tgt = data['translated_highlight']
                
                d = {"translation": {"src": en, "tgt": tgt, 'prefix': 'ringkasan: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                d = {"translation": {"src": ms, "tgt": tgt, 'prefix': 'ringkasan: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')

test-translated-cnn-daily.jsonl


11490it [00:31, 369.96it/s]


train-translated-cnn-daily.jsonl


287219it [14:33, 329.00it/s]


In [28]:
with open('test.json', 'w') as fopen_jsonl:
    for f in ['val-translated-cnn-daily.jsonl']:
        print(f)
        with open(f) as fopen:
            for l in tqdm(fopen):
                data = json.loads(l)
                en = cut_article(data['article'])
                ms = cut_article(data['translated_article'])
                tgt = data['translated_highlight']
                
                d = {"translation": {"src": en, "tgt": tgt, 'prefix': 'ringkasan: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                d = {"translation": {"src": ms, "tgt": tgt, 'prefix': 'ringkasan: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')

val-translated-cnn-daily.jsonl


13368it [00:46, 287.36it/s]


In [29]:
!head -n 4000 test.json > test-4k.json

In [30]:
!cp ../translation/run_t5.py .